<h3>This notebook contains code that will augment the images before the training. It does the following:</h3>

1. Add shapes such as transparent ellipses onto the images to simulate shadows.
2. "Gradient shadowing" - the purpose is the same as in the first point but with using a gradient over the whole picture
3. Blurs the images to simulate bad camera placement.
4. Adds various artifacts to simulate dirty lens, dirty skin etc.

In [40]:
import random
import os

import numpy as np
import PIL.Image
import PIL as pil

from enum import Enum
from PIL import Image, ImageDraw, ImageChops, ImageFilter
from typing import Tuple
from dataclasses import dataclass

<h3>Transparent ellipses</h3>

In [41]:
def crop_image(image: pil.Image.Image) -> Tuple[int, int]:
    width, height = image.size
    vertical_jitter = random.randint(int(height * .3), int(height * .4))
    horizontal_jitter = random.randint(int(width * .3), int(width * .4))

    return vertical_jitter, horizontal_jitter

In [42]:
def get_shadow_image(height: int, width: int) -> pil.Image.Image:
    img = pil.Image.new('RGBA', (width, height), (0, 0, 0, 0))
    draw = pil.ImageDraw.Draw(img)
    center_x = width // 2
    center_y = height // 2

    for y in range(height):
        for x in range(width):
            dist_x = abs(x - center_x) / center_x
            dist_y = abs(y - center_y) / center_y
            dist = (dist_x**2 + dist_y**2)
            alpha = 191 - int(dist * 191)

            draw.point((x, y), fill=(0, 0, 0, alpha))

    return img

In [43]:
def apply_point_shadow(box, original_image: pil.Image.Image, shadow: pil.Image.Image) -> pil.Image.Image:
    padded = pil.Image.new('RGBA', original_image.size, (255, 255, 255, 0))

    padded.paste(shadow, box)

    image_with_shadow = pil.Image.alpha_composite(original_image, padded)

    return image_with_shadow

In [44]:
def get_shadow_positions(original_image: pil.Image.Image, shadow: pil.Image.Image) -> list[Tuple[int, int]]:
    step = 60
    step_x = original_image.size[0] // step
    step_y = original_image.size[1] // step
    margin_v = shadow.size[1] // 2
    margin_h = shadow.size[0] // 2
    moves_to_the_right = [(-margin_v, step * idx - margin_h) for idx in range(step_x)]
    moves_to_the_bottom = [(step * idx - margin_v, (step_x - 1) * step - margin_h) for idx in range(step_y)]
    moves_to_the_left = [((step_y - 1) * step - margin_v, step * idx - margin_h) for idx in reversed(range(step_x))]
    moves_to_the_top = [(step * idx - margin_v, -margin_h) for idx in reversed(range(step_y))]
    shadow_positions = moves_to_the_right + moves_to_the_bottom + moves_to_the_left + moves_to_the_top
    shadow_positions = [move for move in shadow_positions if bool(random.getrandbits(1))]

    return shadow_positions

In [45]:
@dataclass
class ShadowData:
    shadow_img: PIL.Image.Image
    shadow_positions: list[Tuple[int, int]]


def get_shadow_data(original_image: pil.Image.Image) -> ShadowData:
    min_ellipse_size = int(min(original_image.size) * .9)
    max_ellipse_size = int(min(original_image.size) * 1.2)

    while True:
        width = random.randint(min_ellipse_size, max_ellipse_size)
        height = random.randint(min_ellipse_size, max_ellipse_size)
        start_x = random.randint(-width // 2, original_image.size[0] - width // 2)
        start_y = random.randint(-height // 2, original_image.size[1] - height // 2)
        end_x = start_x + width
        end_y = start_y + height
        inside_image_area = max(0, min(end_x, original_image.size[0]) - max(0, start_x)) * max(0, min(end_y, original_image.size[1]) - max(0, start_y))
        total_area = width * height

        if (inside_image_area / total_area) >= 0.4:
            break

    shadow = get_shadow_image(height, width)
    shadow_positions = get_shadow_positions(original_image, shadow)

    return ShadowData(shadow, shadow_positions)

In [46]:
get_paths = lambda path: [f'{os.path.join(root, file)}' for root, dirs, files in os.walk(path) for file in files]
original_path = os.path.join('..', '..', 'data', 'images_original')
image_paths = get_paths(original_path)

In [47]:
# prevent accidental run of this cell by checking if there are files in the augmented dir
def directory_contains_files(directory_path):
    # Get the list of items in the directory
    items = os.listdir(directory_path)

    # Check if any item in the directory is a file
    for item in items:
        item_path = os.path.join(directory_path, item)
        if os.path.isfile(item_path):
            return True

    return False


point_shadow_augmentation_path = os.path.join('..', '..', 'data', 'images_point_shadows')

if not os.path.exists(point_shadow_augmentation_path):
    os.makedirs(point_shadow_augmentation_path)
if directory_contains_files(point_shadow_augmentation_path):
    print(f'The directory {point_shadow_augmentation_path} already has some images in it.')
else:
    def point_shadow_worker(image_path: str) -> None:
        image = Image.open(image_path).convert('RGBA')
        path_parts = image_path.split(os.sep)
        name_parts = path_parts[-1].split('.')
        shadow_data = get_shadow_data(image)
        copy_counter = 0

        for move_y, move_x in shadow_data.shadow_positions:
            copy = apply_point_shadow((move_x, move_y), image, shadow_data.shadow_img)
            copy.convert('RGB').save(f'{point_shadow_augmentation_path}{os.sep}{name_parts[0]}_point_shadow_{copy_counter}.jpg')

            copy_counter += 1


    for image_path in image_paths:
        point_shadow_worker(image_path)

<h3>Gradient shadowing</h3>

In [48]:
class GradientDirection(Enum):
    TOP_TO_BOTTOM = 0
    BOTTOM_TO_TOP = 1
    LEFT_TO_RIGHT = 2
    RIGHT_TO_LEFT = 3
    TOP_LEFT_TO_BOTTOM_RIGHT = 4
    TOP_RIGHT_TO_BOTTOM_LEFT = 5
    BOTTOM_LEFT_TO_TOP_RIGHT = 6
    BOTTOM_RIGHT_TO_TOP_LEFT = 7


def apply_gradient_shadow(
    image: PIL.Image.Image,
    direction: GradientDirection,
    gradient_length=0.5) -> PIL.Image.Image:

    img = image.copy()
    gradient = Image.new('L', (img.width, img.height))
    x = np.linspace(0, gradient_length, img.width)
    y = np.linspace(0, gradient_length, img.height)
    X, Y = np.meshgrid(x, y)

    if direction == GradientDirection.TOP_TO_BOTTOM:
        Z = Y
    elif direction == GradientDirection.BOTTOM_TO_TOP:
        Z = np.flipud(Y)
    elif direction == GradientDirection.LEFT_TO_RIGHT:
        Z = X
    elif direction == GradientDirection.RIGHT_TO_LEFT:
        Z = np.fliplr(X)
    elif direction == GradientDirection.TOP_LEFT_TO_BOTTOM_RIGHT:
        Z = np.sqrt(X**2 + Y**2)
    elif direction == GradientDirection.TOP_RIGHT_TO_BOTTOM_LEFT:
        Z = np.sqrt(np.fliplr(X)**2 + Y**2)
    elif direction == GradientDirection.BOTTOM_LEFT_TO_TOP_RIGHT:
        Z = np.sqrt(X**2 + np.flipud(Y)**2)
    elif direction == GradientDirection.BOTTOM_RIGHT_TO_TOP_LEFT:
        Z = np.sqrt(np.fliplr(X)**2 + np.flipud(Y)**2)

    gradient_data = np.floor((255 * Z / Z.max())).astype(np.uint8)

    gradient.putdata(gradient_data.flatten())

    return ImageChops.multiply(img, gradient.convert('RGBA'))

In [49]:
gradient_shadow_augmentation_path = os.path.join('..', '..', 'data', 'images_gradient_shadows')

if not os.path.exists(gradient_shadow_augmentation_path):
    os.makedirs(gradient_shadow_augmentation_path)
if directory_contains_files(gradient_shadow_augmentation_path):
    print(f'The directory {gradient_shadow_augmentation_path} already has some images in it.')
else:
    def gradient_shadow_worker(image_path: str) -> None:
        image = Image.open(image_path).convert('RGBA')
        path_parts = image_path.split(os.sep)
        name_parts = path_parts[-1].split('.')
        copy_counter = 0

        for direction in GradientDirection:
            copy = apply_gradient_shadow(image, direction)

            copy.convert('RGB').save(os.path.join(gradient_shadow_augmentation_path, f'{name_parts[0]}_gradient_shadow_{copy_counter}.jpg'))

            copy_counter += 1


    for image_path in image_paths:
        gradient_shadow_worker(image_path)

<h3>Blurring</h3>

In [50]:
def apply_blur(image: PIL.Image.Image, radius: int) -> PIL.Image.Image:
    return image.filter(ImageFilter.GaussianBlur(radius=radius))

In [51]:
blurring_augmentation_path = os.path.join('..', '..', 'data', 'images_blurred')

if not os.path.exists(blurring_augmentation_path):
    os.makedirs(blurring_augmentation_path)
if directory_contains_files(blurring_augmentation_path):
    print(f'The directory {blurring_augmentation_path} already has some images in it.')
else:
    def blur_worker(image_path: str) -> None:
        image = Image.open(image_path).convert('RGBA')
        path_parts = image_path.split(os.sep)
        name_parts = path_parts[-1].split('.')
        copy_counter = 0

        for blur_radius in [3, 4, 5]:
            copy = apply_blur(image, blur_radius)

            copy.convert('RGB').save(os.path.join(blurring_augmentation_path, f'{name_parts[0]}_blurred_{copy_counter}.jpg'))

            copy_counter += 1


    for image_path in image_paths:
        blur_worker(image_path)

<h3>Dirty lens</h3>

In [52]:
def apply_dirty_lens_effect(image: PIL.Image.Image) -> PIL.Image.Image:
    dirty_lens = Image.new('RGBA', image.size, (0, 0, 0, 0))

    draw = ImageDraw.Draw(dirty_lens)

    for _ in range(5):
        pos_x = random.randint(0, image.width)
        pos_y = random.randint(0, image.height)
        radius = random.randint(20, 60)
        transparency = random.randint(50, 120)

        draw.ellipse([(pos_x-radius, pos_y-radius), (pos_x+radius, pos_y+radius)], fill=(125, 105, 83, transparency))

    dirty_image = Image.alpha_composite(image.convert('RGBA'), dirty_lens)

    return dirty_image

In [53]:
dirty_lens_augmentation_path = os.path.join('..', '..', 'data', 'images_dirty_lens')

if not os.path.exists(dirty_lens_augmentation_path):
    os.makedirs(dirty_lens_augmentation_path)
if directory_contains_files(dirty_lens_augmentation_path):
    print(f'The directory {dirty_lens_augmentation_path} already has some images in it.')
else:
    def dirty_lens_worker(image_path: str) -> None:
        image = Image.open(image_path).convert('RGBA')
        path_parts = image_path.split(os.sep)
        name_parts = path_parts[-1].split('.')
        copy = apply_dirty_lens_effect(image)

        copy.convert('RGB').save(os.path.join(dirty_lens_augmentation_path, f'{name_parts[0]}_dirtry_lens.jpg'))


    for image_path in image_paths:
        dirty_lens_worker(image_path)